# 国家统计局
http://data.stats.gov.cn/ 月度统计数据

- 尽可能汽车相关
- 尽可能是数值而不是比值
- 尽可能统计到 18.2
    - 否则分别给出不同时长的特征表
- 如果统一缺少月份数据使用前后两月的平均...
- ...

> 然后生成专门的特征表生成 SQL 以便回填到 MC 中


## 当前数据文件

```
raw/_macroeconomic/
    2种数值-1802.csv
    3种数值-1801.csv
    6种数值--1802.csv (每年1月缺少)
```

## pd 清洗/转换

In [1]:
import pandas as pd
print(pd.__version__)
import numpy as np
import matplotlib.pyplot as plt
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn

import datetime


0.22.0


In [4]:
ls -1 ../../raw/_macroeconomic/*.csv

../../raw/_macroeconomic/2种数值-1802.csv
../../raw/_macroeconomic/3种数值-1801.csv
../../raw/_macroeconomic/6种数值--1802.csv


## 两组无缺数值

In [5]:
df = pd.read_csv('../../raw/_macroeconomic/2种数值-1802.csv')
df.head()

,时间,非制造业商务活动指数(%),从业人员指数(%)
0,2018年2月,54.4,49.6
1,2018年1月,55.3,49.4
2,2017年12月,55.0,49.3
3,2017年11月,54.8,49.2
4,2017年10月,54.3,49.4


In [6]:
_date = df['时间']

In [7]:
_date[0]

'2018年2月'

In [8]:
_day = datetime.datetime.strptime(_date[0] , "%Y年%m月")

In [9]:
_day.strftime('%Y%m')

'201802'

In [10]:
df['时间'] = df['时间'].apply(lambda x:datetime.datetime.strptime(x , "%Y年%m月").strftime('%Y%m'))

In [11]:
df['时间'][-1:]

73    201201
Name: 时间, dtype: object

In [12]:
len(df['时间'])

74

In [13]:
df.tail()

,时间,非制造业商务活动指数(%),从业人员指数(%)
69,201205,55.2,51.3
70,201204,56.1,52.1
71,201203,58.0,52.3
72,201202,57.3,53.8
73,201201,55.7,53.1


In [14]:
df.isnull().any()

时间               False
非制造业商务活动指数(%)    False
从业人员指数(%)        False
dtype: bool

In [15]:
redf = df.rename(columns={'时间':'month'
                  , '非制造业商务活动指数(%)':'1802a'
                  , '从业人员指数(%)':'1802b'

                  })

### SQL 生成
对应数据表定义, 值定义...

In [68]:
tpl_SQL = '''-- {mc_tname}
-- 提取合理数值表
-- <= bitb:raw/_macroeconomic/{mc_res}
-- => {mc_tname}
-- 
-- 如果存在就嗯哼
DROP TABLE IF EXISTS 
    {mc_tname}
;
-- 
-- 就地创建
CREATE TABLE IF NOT EXISTS 
    {mc_tname} (month string 
         ,{mc_fiels}
        )
;
-- 
-- 逐一嗯哼
INSERT INTO TABLE 
    {mc_tname} 
VALUES 
     {mc_vars}
;
-- 
-- 视验
SELECT * FROM {mc_tname}
LIMIT 10
;

'''

_stuff = {'mc_tname': 'ycmecon_econ2stats_1802'
          , 'mc_res':'2种数值-1802.csv'
          , 'mc_fiels': ''
          , 'mc_vars': ''
         }
exp_SQL = "../../src/SQL/DAMA_0310_feature_econ2stats_1802.sql"


In [17]:
#redf.columns
#','.join(['%s bigint'%k for k in redf.columns[1:]])
_stuff['mc_fiels'] = '\n,'.join(['%s bigint'%k for k in redf.columns[1:]])

In [20]:
#print(type(redf.iloc[5,:]))
#for v in redf.iloc[5,:]:    print(v)
#','.join('%s'%v for v in redf.iloc[5,1:])
#redf.iloc[5,:][0]

In [21]:
_var = []
for row in range(len(redf)):
    _var.append("('{}',{})".format(
                redf.iloc[row,:][0]
                , ','.join('%s'%v for v in redf.iloc[row,1:])
                ))
#print(_var)
_stuff['mc_vars'] = '\n,'.join(_var)

In [22]:
print("exp==>%s"%exp_SQL)
open(exp_SQL,'w').write(tpl_SQL.format(**_stuff))

exp==>../../src/SQL/DAMA_0310_feature_econ2stats_1802.sql


2073

In [23]:
len(redf)

74

## 3组无缺数值~1801

In [32]:
df = pd.read_csv('../../raw/_macroeconomic/3种数值-1801.csv')
df.head()

,时间,固定长途电话通话时长(含IP)_当期值(万分钟),快递量_当期值(万件),沿海规模以上港口货物吞吐量_当期值(万吨)
0,2018年1月,244053.9,398613.3,76731.0
1,2017年12月,250458.9,420037.7,67605.0
2,2017年11月,257606.8,471490.3,70250.0
3,2017年10月,230970.3,374771.5,72275.0
4,2017年9月,259242.9,360605.6,71404.0


In [33]:
df['时间'] = df['时间'].apply(lambda x:datetime.datetime.strptime(x , "%Y年%m月").strftime('%Y%m'))
#df['时间'][-1:]
len(df['时间'])

73

In [36]:
df.isnull().any()

时间                          False
固定长途电话通话时长(含IP)_当期值(万分钟)    False
快递量_当期值(万件)                 False
沿海规模以上港口货物吞吐量_当期值(万吨)        True
dtype: bool

In [37]:
redf = df.rename(columns={'时间':'month'
                  , '固定长途电话通话时长(含IP)_当期值(万分钟)':'1801a'
                  , '快递量_当期值(万件)':'1801b'
                  , '沿海规模以上港口货物吞吐量_当期值(万吨)':'1801c'
                  })

### 增补缺值
~ 用前后数值的平均来回填

In [39]:
redf.isnull().any()

month    False
1801a    False
1801b    False
1801c     True
dtype: bool

In [65]:
for i in range(len(redf['1801c'])):
    #print(redf['1801c'][i])
    if np.isnan(redf['1801c'][i]):# == np.nan:
        _mean = (redf['1801c'][i-1]+redf['1801c'][i+1])/2
        print('NaN {} {} ->{}'.format(
                    redf['1801c'][i-1]
                    , redf['1801c'][i+1]
                    , _mean
                    ))
        redf['1801c'][i] = _mean


NaN 70803.0 67678.0 ->69240.5
NaN 67573.0 63126.0 ->65349.5
NaN 68101.0 64220.0 ->66160.5
NaN 65186.0 60599.0 ->62892.5
NaN 60001.0 56220.0 ->58110.5


In [67]:
redf.isnull().any()

month    False
1801a    False
1801b    False
1801c    False
dtype: bool

### SQL 生成
对应数据表定义, 值定义...

In [70]:
_stuff = {'mc_tname': 'ycmecon_econ3stats_1801'
          , 'mc_res':'3种数值-1801.csv'
          , 'mc_fiels': ''
          , 'mc_vars': ''
         }
exp_SQL = "../../src/SQL/DAMA_0310_feature_econ3stats_1801.sql"
_stuff['mc_fiels'] = '\n,'.join(['%s bigint'%k for k in redf.columns[1:]])
_var = []
for row in range(len(redf)):
    _var.append("('{}',{})".format(
                redf.iloc[row,:][0]
                , ','.join('%s'%v for v in redf.iloc[row,1:])
                ))
#print(_var)
_stuff['mc_vars'] = '\n,'.join(_var)
print("exp==>%s"%exp_SQL)
open(exp_SQL,'w').write(tpl_SQL.format(**_stuff))

exp==>../../src/SQL/DAMA_0310_feature_econ3stats_1801.sql


3208

In [69]:
len(redf)

73

## 16组有缺数值~1712

In [93]:
df = pd.read_csv('../../raw/_macroeconomic/16种数值-1712.csv')
df.head()

,时间,办公楼竣工面积_累计增长(%),渔业固定资产投资额_累计增长(%),畜牧业固定资产投资额_累计增长(%),林业固定资产投资额_累计增长(%),农业固定资产投资额_累计增长(%),电信、广播电视和卫星传输服务固定资产投资额_累计增长(%),信息传输、软件和信息技术服务业固定资产投资额_累计增长(%),水的生产和供应业固定资产投资额_累计增长(%),航空运输业固定资产投资额_累计增长(%),管道运输业固定资产投资额_累计增长(%),仓储业固定资产投资额_累计增长(%),科学研究、技术服务和地质勘查业固定资产投资额_累计增长(%),公共设施管理业固定资产投资额_累计增长(%),环境管理业固定资产投资额_累计增长(%),公共管理和社会组织固定资产投资额_累计增长(%),文化艺术业固定资产投资额_累计增长(%)
0,2017年12月,10.4,12.7,6.4,2.8,16.4,-5.2,12.8,11.6,8.5,32.5,-0.4,9.4,21.8,23.9,-2.0,12.6
1,2017年11月,28.0,13.4,4.7,3.2,16.6,-4.3,15.2,12.2,4.7,37.3,1.5,8.7,23.3,23.6,4.1,11.9
2,2017年10月,36.5,15.8,7.0,5.4,17.6,-5.7,13.4,12.9,2.7,41.9,3.3,8.7,23.4,24.1,1.8,11.2
3,2017年9月,36.1,17.6,5.0,5.5,16.2,-6.9,10.6,12.8,-3.4,53.5,4.6,10.0,23.7,25.0,4.0,13.2
4,2017年8月,43.7,17.3,5.9,7.1,16.1,-8.1,11.3,14.6,-8.9,70.9,4.2,11.8,24.3,28.2,3.0,15.0


In [94]:
df['时间'] = df['时间'].apply(lambda x:datetime.datetime.strptime(x , "%Y年%m月").strftime('%Y%m'))
#df['时间'][-1:]
len(df['时间'])

72

In [95]:
df.isnull().any()

时间                                False
办公楼竣工面积_累计增长(%)                    True
渔业固定资产投资额_累计增长(%)                  True
畜牧业固定资产投资额_累计增长(%)                 True
林业固定资产投资额_累计增长(%)                  True
农业固定资产投资额_累计增长(%)                  True
电信、广播电视和卫星传输服务固定资产投资额_累计增长(%)      True
信息传输、软件和信息技术服务业固定资产投资额_累计增长(%)     True
水的生产和供应业固定资产投资额_累计增长(%)            True
航空运输业固定资产投资额_累计增长(%)               True
管道运输业固定资产投资额_累计增长(%)               True
仓储业固定资产投资额_累计增长(%)                 True
科学研究、技术服务和地质勘查业固定资产投资额_累计增长(%)     True
公共设施管理业固定资产投资额_累计增长(%)             True
环境管理业固定资产投资额_累计增长(%)               True
公共管理和社会组织固定资产投资额_累计增长(%)           True
文化艺术业固定资产投资额_累计增长(%)               True
dtype: bool

In [96]:
redf=df.rename(columns={'时间':'month'
        , '办公楼竣工面积_累计增长(%)':'1712a'
        , '渔业固定资产投资额_累计增长(%)':'1712b'
        , '畜牧业固定资产投资额_累计增长(%)':'1712c'
        , '林业固定资产投资额_累计增长(%)':'1712d'
        , '农业固定资产投资额_累计增长(%)':'1712e'
        , '电信、广播电视和卫星传输服务固定资产投资额_累计增长(%)':'1712f'
        , '信息传输、软件和信息技术服务业固定资产投资额_累计增长(%)':'1712g'
        , '水的生产和供应业固定资产投资额_累计增长(%)':'1712h'
        , '航空运输业固定资产投资额_累计增长(%)':'1712i'
        , '管道运输业固定资产投资额_累计增长(%)':'1712j'
        , '仓储业固定资产投资额_累计增长(%)':'1712k'
        , '科学研究、技术服务和地质勘查业固定资产投资额_累计增长(%)':'1712kk'
        , '公共设施管理业固定资产投资额_累计增长(%)':'1712m'
        , '环境管理业固定资产投资额_累计增长(%)':'1712n'
        , '公共管理和社会组织固定资产投资额_累计增长(%)':'1712o'
        , '文化艺术业固定资产投资额_累计增长(%)':'1712p'
        })


### 增补缺值
~ 用前后数值的平均来回填

In [97]:
redf.isnull().any()

month     False
1712a      True
1712b      True
1712c      True
1712d      True
1712e      True
1712f      True
1712g      True
1712h      True
1712i      True
1712j      True
1712k      True
1712kk     True
1712m      True
1712n      True
1712o      True
1712p      True
dtype: bool

In [98]:
def _fix4nan2mon(df1s):
    for i in range(len(df1s)):
        #print(redf['1801c'][i])
        if np.isnan(df1s[i]):# == np.nan:
            _mean = (df1s[i-1] + df1s[i+1])/2
            print('NaN {} {} ->{}'.format(
                        df1s[i-1]
                        , df1s[i+1]
                        , _mean
                        ))
            df1s[i] = _mean
    return df1s


In [99]:
for col in redf.columns[1:]:
    #print(col)
    redf[col] = _fix4nan2mon(redf[col])
    
redf.isnull().any()

NaN 25.7 6.1 ->15.899999999999999
NaN 6.6 8.8 ->7.7
NaN 36.4 12.7 ->24.549999999999997
NaN 14.0 20.5 ->17.25
NaN 86.8 2.1 ->44.449999999999996
NaN 39.2 21.8 ->30.5
NaN 26.4 16.2 ->21.299999999999997
NaN 14.8 17.2 ->16.0
NaN 11.8 30.3 ->21.05
NaN 63.1 41.29 ->52.195
NaN 16.8 10.1 ->13.45
NaN 15.7 19.7 ->17.7
NaN 20.8 35.7 ->28.25
NaN 17.8 38.7 ->28.25
NaN 40.1 32.28 ->36.19
NaN -14.1 10.6 ->-1.75
NaN 38.5 23.5 ->31.0
NaN 38.4 18.5 ->28.45
NaN 3.2 33.9 ->18.55
NaN 41.2 14.99 ->28.095000000000002
NaN 24.9 30.4 ->27.65
NaN 44.0 45.7 ->44.85
NaN 49.4 40.6 ->45.0
NaN 30.5 28.1 ->29.3
NaN 30.3 40.53 ->35.415
NaN -2.4 8.3 ->2.95
NaN 15.9 18.4 ->17.15
NaN 38.6 25.6 ->32.1
NaN -4.2 8.2 ->1.9999999999999996
NaN -22.7 15.34 ->-3.6799999999999997
NaN 13.4 14.5 ->13.95
NaN 25.4 34.5 ->29.95
NaN 36.8 38.6 ->37.7
NaN 22.4 19.5 ->20.95
NaN 2.9 30.63 ->16.765
NaN 26.0 20.5 ->23.25
NaN 50.6 30.7 ->40.65
NaN 29.6 16.8 ->23.200000000000003
NaN 4.2 28.1 ->16.150000000000002
NaN 26.9 18.39 ->22.645
NaN 69.2 

month     False
1712a     False
1712b     False
1712c     False
1712d     False
1712e     False
1712f     False
1712g     False
1712h     False
1712i     False
1712j     False
1712k     False
1712kk    False
1712m     False
1712n     False
1712o     False
1712p     False
dtype: bool

### SQL 生成
对应数据表定义, 值定义...

In [100]:
_stuff = {'mc_tname': 'ycmecon_econ16stats_1712'
          , 'mc_res':'16种数值-1712.csv'
          , 'mc_fiels': ''
          , 'mc_vars': ''
         }
exp_SQL = "../../src/SQL/DAMA_0310_feature_econ16stats_1712.sql"
_stuff['mc_fiels'] = '\n,'.join(["%s bigint"%k for k in redf.columns[1:]])
_var = []
for row in range(len(redf)):
    _var.append("('{}',{})".format(
                redf.iloc[row,:][0]
                , ','.join('%s'%v for v in redf.iloc[row,1:])
                ))
#print(_var)
_stuff['mc_vars'] = '\n,'.join(_var)
print("exp==>%s"%exp_SQL)
open(exp_SQL,'w').write(tpl_SQL.format(**_stuff))

exp==>../../src/SQL/DAMA_0310_feature_econ16stats_1712.sql


7383